In [ ]:
# #uncomment if you do not have requests and websocket-client (version 0.49 and above) installed
# #Install requests package in a current Jupyter kernal

# import sys
# !{sys.executable} -m pip install requests
# !{sys.executable} -m pip install websocket-client

In [ ]:
# import required libraries

import sys
import time
import getopt
import socket
import json
import websocket
import threading
import os
from threading import Thread, Event

In [ ]:
# TREP connection variables

hostname = '127.0.0.1'
port = '15000'
user = 'root'
app_id = '256'
position = socket.gethostbyname(socket.gethostname())
login_id = 1

In [ ]:
# TRCC contribution variables

post_item_name = 'CONTRIBUTION_RIC'
service_name = 'TRCC'

post_id = 1
bid_value = 34.25
ask_value = 35.48
primact_1_value = 116.50

In [ ]:
# WebSocket connections Variables

web_socket_app = None
web_socket_open = False

In [ ]:
# Create JSON Off-Stream Post message and sends it to ADS server.

def send_market_price_post(ws):
    """ Send a post message contains a market-price content to TRCC """

    """ Contribution fields """
    contribution_fields = {
        "BID": bid_value,
        "ASK": ask_value,
        "PRIMACT_1": primact_1_value
    }

    """ OMM Post msg Key """
    mp_post_key = {
        "Name": post_item_name,
        "Service": service_name
    }

    """ OMM Post Payload """
    contribution_payload_json = {
        "ID": 0,
        "Type": "Update",
        "Domain": "MarketPrice",
        "Fields": contribution_fields,
        "Key": {}
    }

    """ OMM Off-Stream Post message """
    mp_post_json_offstream = {
        "Domain": "MarketPrice",
        "Ack": True,
        "PostID": post_id,
        "PostUserInfo": {
            "Address": position,
            "UserID": int(app_id)
        },
        "Key": {},
        "Message": {},
        "Type": "Post",
        "ID": login_id #Off-Stream post, sending the Post message on the Loing stream.
    }

    contribution_payload_json["Key"] = mp_post_key
    mp_post_json_offstream["Key"] = mp_post_key
    mp_post_json_offstream["Message"] = contribution_payload_json

    ws.send(json.dumps(mp_post_json_offstream))
    print("SENT:")
    print(json.dumps(mp_post_json_offstream, sort_keys=True, indent=2, separators=(',', ':')))


In [ ]:
# JSON-OMM Login and Process messages

def process_message(ws, message_json):  # Process all incoming messages.
    """ Parse at high level and output JSON of message """
    message_type = message_json['Type']

    if message_type == "Refresh":
        if 'Domain' in message_json:
            message_domain = message_json['Domain']
            if message_domain == "Login":
                process_login_response(ws, message_json)
    elif message_type == "Ping":
        pong_json = {'Type': 'Pong'}
        ws.send(json.dumps(pong_json))
        print("SENT:")
        print(json.dumps(pong_json, sort_keys=True,
                         indent=2, separators=(',', ':')))
        
# Process incoming Login Refresh Response message.
def process_login_response(ws, message_json):
    """ Send Off-Stream Post """
    print("Sending Off-Stream Post to TREP Server")
    send_market_price_post(ws)    
    
# Create JSON Login request message and sends it to ADS server.
def send_login_request(ws):
    """ Generate a login request from command line data (or defaults) and send """
    login_json = {
        'ID': login_id,
        'Domain': 'Login',
        'Key': {
            'Name': '',
            'Elements': {
                'ApplicationId': '',
                'Position': ''
            }
        }
    }

    login_json['Key']['Name'] = user
    login_json['Key']['Elements']['ApplicationId'] = app_id
    login_json['Key']['Elements']['Position'] = position

    ws.send(json.dumps(login_json))
    print("SENT:")
    print(json.dumps(login_json, sort_keys=True, indent=2, separators=(',', ':')))

In [ ]:
# WebSocket connection process

def on_message(ws, message):
    """ Called when message received, parse message into JSON for processing """
    print("RECEIVED: ")
    message_json = json.loads(message)
    print(json.dumps(message_json, sort_keys=True, indent=2, separators=(',', ':')))

    for singleMsg in message_json:
        process_message(ws, singleMsg)
        
def on_error(ws, error):
    """ Called when websocket error has occurred """
    print(error)
    
def on_close(ws):
    """ Called when websocket is closed """
    global web_socket_open
    print("WebSocket Closed")
    web_socket_open = False
    
def on_open(ws):
    """ Called when handshake is complete and websocket is open, send login """

    print("WebSocket successfully connected!")
    global web_socket_open
    web_socket_open = True
    send_login_request(ws)

if __name__ == "__main__":
    # Start websocket handshake
    ws_address = "ws://{}:{}/WebSocket".format(hostname, port)
    print("Connecting to WebSocket " + ws_address + " ...")
    web_socket_app = websocket.WebSocketApp(ws_address, header=['User-Agent: Python'],
                                            on_message=on_message,
                                            on_error=on_error,
                                            on_close=on_close,
                                            subprotocols=['tr_json2'])
    web_socket_app.on_open = on_open

    # Event loop
    #wst = threading.Thread(target=web_socket_app.run_forever)
    wst = threading.Thread(target=web_socket_app.run_forever, kwargs={'sslopt': {'check_hostname': False}})
    wst.start()

    time.sleep(10)
    web_socket_app.close()
